# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("./output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,13.05,85,3,9.62,NZ,2023-11-13 23:03:57
1,1,grytviken,-54.2811,-36.5092,2.53,98,100,3.27,GS,2023-11-13 23:04:12
2,2,talayuela,39.9870,-5.6098,14.44,89,63,1.08,ES,2023-11-13 23:04:12
3,3,klyuchi,52.2667,79.1667,4.01,89,100,4.98,RU,2023-11-13 23:04:13
4,4,olonkinbyen,70.9221,-8.7187,-2.21,69,95,2.89,SJ,2023-11-13 23:04:13


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_1=city_data_df.hvplot.points(
    "Lng","Lat",geo=True,tiles='OSM',size="Humidity", color="City",alpha=0.5
)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities=city_data_df[
        (city_data_df['Humidity']>40) & (city_data_df['Humidity']<80) & 
        (city_data_df['Max Temp']>23) &(city_data_df['Max Temp']<33) &(city_data_df['Cloudiness']<3
                                                                       )
]

# Drop any rows with null values
ideal_cities.dropna()

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,lompoc,34.6391,-120.4579,23.21,64,0,5.14,US,2023-11-13 23:04:14
82,82,cabo san lucas,22.8909,-109.9124,31.45,61,0,3.09,MX,2023-11-13 23:04:28
102,102,port mathurin,-19.6833,63.4167,24.96,78,0,1.19,MU,2023-11-13 23:04:32
217,217,cockburn town,21.4612,-71.1419,28.00,77,1,8.31,TC,2023-11-13 23:04:01
352,352,puerto boyaca,5.9758,-74.5885,27.65,79,2,0.91,CO,2023-11-13 23:05:24
441,441,grand gaube,-20.0064,57.6608,24.10,73,1,1.34,MU,2023-11-13 23:05:43
482,482,laguna,38.4210,-121.4238,23.39,43,0,1.54,US,2023-11-13 23:05:49


In [10]:
print(len(ideal_cities))

7


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_cities[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,lompoc,US,34.6391,-120.4579,64,
82,cabo san lucas,MX,22.8909,-109.9124,61,
102,port mathurin,MU,-19.6833,63.4167,78,
217,cockburn town,TC,21.4612,-71.1419,77,
352,puerto boyaca,CO,5.9758,-74.5885,79,
441,grand gaube,MU,-20.0064,57.6608,73,
482,laguna,US,38.4210,-121.4238,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
limit=20
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":limit
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
cabo san lucas - nearest hotel: Comfort Rooms
port mathurin - nearest hotel: Escale Vacances
cockburn town - nearest hotel: The Salt Raker Inn
puerto boyaca - nearest hotel: Hotel Gran Esquina
grand gaube - nearest hotel: Hotel Lux- Grand Gaube
laguna - nearest hotel: Holiday Inn Express & Suites


,City,Country,Lat,Lng,Humidity,Hotel Name
11,lompoc,US,34.6391,-120.4579,64,Embassy Suites by Hilton Lompoc Central Coast
82,cabo san lucas,MX,22.8909,-109.9124,61,Comfort Rooms
102,port mathurin,MU,-19.6833,63.4167,78,Escale Vacances
217,cockburn town,TC,21.4612,-71.1419,77,The Salt Raker Inn
352,puerto boyaca,CO,5.9758,-74.5885,79,Hotel Gran Esquina
441,grand gaube,MU,-20.0064,57.6608,73,Hotel Lux- Grand Gaube
482,laguna,US,38.4210,-121.4238,43,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map=hotel_df.hvplot.points(
    "Lng","Lat",geo=True,tiles='OSM',frame_width=600,frame_height=400,size="Humidity", color="City",hover_cols=["Hotel Name", "Country"],alpha=0.5
)
# hotel_map=hotel_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",frame_width=600,frame_height=400, 
# scale=2,color="city",hover_cols=["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)